In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

- `nn.Module`의 하위클래스로 정의

In [2]:
class NeuralNetwork(nn.Module):

    def __init__(self):

        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


- forward를 호출하지 않아도 모델에 입력값을 넣어주면 백그라운드 연산을 통해 결과값이 반환된다

In [4]:
X = torch.rand(1,28,28)
logits = model(X)
print(logits.shape)

torch.Size([1, 10])


In [5]:
prob = nn.Softmax(dim=1)(logits)
pred = prob.argmax(dim=1)
print(pred)

tensor([6])


## autograd

In [6]:
import torch
from torch.nn.functional import binary_cross_entropy_with_logits

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5,3,requires_grad=True)
b = torch.randn(3,requires_grad=True)

print(w)
print(b)

tensor([[-0.4837,  0.0933,  0.9513],
        [-1.5629, -1.1491,  0.8234],
        [ 0.3898,  0.5866, -2.6584],
        [-0.0210, -0.0031,  1.6357],
        [ 0.8001,  0.7442, -1.1413]], requires_grad=True)
tensor([-0.8728,  0.0985, -0.1600], requires_grad=True)


In [7]:
z = torch.matmul(x,w) + b
loss = binary_cross_entropy_with_logits(z,y)

In [8]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0493, 0.1972, 0.1220],
        [0.0493, 0.1972, 0.1220],
        [0.0493, 0.1972, 0.1220],
        [0.0493, 0.1972, 0.1220],
        [0.0493, 0.1972, 0.1220]])
tensor([0.0493, 0.1972, 0.1220])


- `requires_grad = True` : 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다
- 순전파만 필요한 경우에 `torch.no_grad()` (`detach()`)블록으로 둘러싸서 연산 추적을 멈춘다

In [9]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [10]:
z = torch.matmul(x, w) + b
print(z.requires_grad)
z = z.detach()
print(z.requires_grad)

True
False
